<a href="https://colab.research.google.com/github/princetyagitech/Computer_Vision/blob/master/Face_Recognition_leaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install face_recognition

     |████████████████████████████████| 100.2MB 112kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=e7b6ddd82fec97f860ee30eaeba9f2641ab555fa08ae8ac4c7f506266c4fdce5
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
import face_recognition
import cv2
import numpy as np



video_capture = cv2.VideoCapture(0)

# Load a sample picture and learn how to recognize it.
cameron_image = face_recognition.load_image_file("/content/cameron.jfif")
cameron_face_encoding = face_recognition.face_encodings(cameron_image)[0]

# Load a second sample picture and learn how to recognize it.
modi_image = face_recognition.load_image_file("/content/modi.jfif")
modi_face_encoding = face_recognition.face_encodings(modi_image)[0]

putin_image = face_recognition.load_image_file("/content/putin.jfif")
putin_face_encoding = face_recognition.face_encodings(putin_image)[0]

trump_image = face_recognition.load_image_file("/content/trump.jpg")
trump_face_encoding = face_recognition.face_encodings(trump_image)[0]

xi_image = face_recognition.load_image_file("/content/xi.jpg")
xi_face_encoding = face_recognition.face_encodings(xi_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    cameron_face_encoding,
    modi_face_encoding,
    putin_face_encoding,
    trump_face_encoding,
    xi_face_encoding
]
known_face_names = [
    "David Cameron",
    "Narendra Modi",
    "Vladimir Putin",
    "Donald Trump",
    "Xi JinPing"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            

            # # If a match was found in known_face_encodings, just use the first one.
            if True in matches:
              first_match_index = matches.index(True)
              name = known_face_names[first_match_index]
            else:
              name = "Unknown"

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        
    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()